In [1]:
!python -m spacy download nl_core_news_sm

     ---------------------------------------- 17.4/17.4 MB 8.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')


In [36]:
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers
from spacy.tokens import DocBin
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import load_dataset
import nl_core_news_sm
import re
import numpy as np
torch.cuda.is_available(), spacy.prefer_gpu()

(True, True)

In [3]:
dataset = load_dataset("dutch_social")


Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])

In [5]:
train = train.sample(frac = 1)
test = test.sample(frac = 1)

In [6]:
train['label'] = train['label'].astype(str)
test['label'] = test['label'].astype(str)

In [7]:
train.head()

,full_text,text_translation,screen_name,description,desc_translation,location,weekofyear,weekday,month,year,...,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern,label
143826,Who came up with this idea of a story.. https:...,Who came up with this idea of ​​a story .. htt...,LeeSpacey,"Focus on MENA & European Terrorism, Internatio...","Focus on MENA and European Terrorism, Internat...",Netherlands,5,6,2,2020,...,52.500170,5.748082,0.0,Flevoland,None,None,False,0.0000,0.00,1
40139,Retail and Chain manager Jan Doldersum reflect...,Retail Chain Manager and Jan Doldersum reflect...,RijkZwaan,"Together with our partners, we actively want t...","Together with our partners, we want to Activel...","De Lier, The Netherlands",25,4,6,2020,...,51.974661,4.253703,0.0,Zuid-Holland,None,None,False,0.0000,0.00,1
113763,Ik kijk niet eens.. alleen maar leugens van @M...,I'm not even .. just lies @MinPres and @hugode...,alindak36,"Zwolle, Assendorp. Al 21 jaar werkzaam bij de,...","Zwolle, Assendorp. 21 years working in the, fa...",None,36,1,9,2020,...,0.000000,0.000000,0.0,False,None,None,False,-0.0500,0.40,1
22040,RT @NiemandsKnegt: Achtergrondartikel in het E...,RT @NiemandsKnegt: Background article in Engli...,Raaskaldotcom,Integer | hubby & papa | master studie | data ...,integer | hubby & dad | master Study | data sc...,None,36,6,9,2020,...,0.000000,0.000000,0.0,False,None,None,False,0.0000,0.00,1
65867,Dit optimistische geluid in deze #Tegenlicht o...,This sound optimistic in this #Tegenlicht a su...,VeraBauman,"Rotterdam, the Netherlands, Europe, World","Rotterdam, the Netherlands, Europe, World",Rotterdam,19,6,5,2020,...,51.922896,4.463173,0.0,Zuid-Holland,None,None,False,0.3375,0.55,2


In [8]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

def remove_username(text):
    return re.sub('@[\w]+','',text)

def remove_retweet(text):
    
    return text.replace("RT : ", "")

In [9]:
train['full_text'] = train['full_text'].apply(lambda x: remove_link(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_link(x))

train['full_text'] = train['full_text'].apply(lambda x: remove_username(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_username(x))

train['full_text'] = train['full_text'].apply(lambda x: remove_retweet(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_retweet(x))

In [10]:
list(train['full_text'])[50:60]

['Hoezo namens de hele bevolking!  #viruswaanzin is domheid troef! #nietnamensmij! Ik doneer plasma om het #coronavirus te hel…',
 'Een Amerikaspecialist die meer reizen binnen Europa gaat aanbieden?Toerisme wordt nooit meer wat het was voor corona,stelt…',
 'Today my son was cornered at school by kids who wanted to “test” him for #Coronavirus just because he is half-Chinese. The…',
 'De grootste crisis sinds de Tweede Wereldoorlog maar niemand weet wie wanneer moet ingrijpen. Goede analyse door …',
 'Wanneer gaan we het over MIS-C hebben?\n\nCDC\nFor Parents: Multisystem Inflammatory Syndrome in Children (MIS-C) associated with…',
 'Kijk op   naar #caringfarmer en  oprichter  over corona, Kipster en kansen:  ',
 'Bij een rit van het ziekenhuis naar revalidatiecentrum met de ambulance stond de familie van een #COVID19 patiënt op veilig…',
 '   Je vroeg naar ebola toch? Dat is overdraagbaar na fysiek contact met een met ebola en echt ziek persoon. Daarin verschillen virussen in eigensch

In [11]:
train_zip = list(zip(train['full_text'], train['label']))

In [12]:
test_zip = list(zip(test['full_text'], test['label']))

In [13]:
!pip install spacy
!pip install spacy-transformers

In [14]:
nlp=spacy.load("nl_core_news_sm")

In [15]:
def document(data):
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='0'):
      doc.cats['0'] = 1
      doc.cats['1'] = 0
      doc.cats['2']  = 0
    elif (label=='1'):
      doc.cats['0'] = 0
      doc.cats['1'] = 1
      doc.cats['2']  = 0
    elif (label=='2'):
      doc.cats['0'] = 0
      doc.cats['1'] = 0
      doc.cats['2']  = 1
    text.append(doc)
  
  return(text)

In [16]:
train_docs = document(train_zip)
doc_bin = DocBin(docs = train_docs)
doc_bin.to_disk("train.spacy")


In [17]:
test_docs = document(test_zip)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")


In [18]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg


[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
start_time = datetime.now()

!python -m spacy train config.cfg --verbose  --gpu-id 0 --output ./output_updated

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

[i] Saving to output directory: output_updated
[i] Using GPU: 0

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.17        0.00    0.00
  0     100          5.20       16.20    0.16
  0     200          4.64       21.23    0.21
  0     300          2.85       24.93    0.25
  0     400          4.49       31.87    0.32
  0     500          3.35       36.70    0.37
  0     600          3.86       41.13    0.41
  0     700          2.45       46.73    0.47
  0     800          2.90       48.05    0.48
  0     900          3.13       51.12    0.51
  0    1000          4.18       54.03    0.54
  0    1100          3.37       56.12    0.56
  0    1200          3.36       56.50    0.57
  0    

[2022-04-10 17:08:09,656] [INFO] Set up nlp object from config
[2022-04-10 17:08:09,660] [DEBUG] Loading corpus from path: valid.spacy
[2022-04-10 17:08:09,661] [DEBUG] Loading corpus from path: train.spacy
[2022-04-10 17:08:09,661] [INFO] Pipeline: ['textcat']
[2022-04-10 17:08:09,663] [INFO] Created vocabulary
[2022-04-10 17:08:09,663] [INFO] Finished initializing nlp object
[2022-04-10 17:09:50,942] [INFO] Initialized pipeline components: ['textcat']
[2022-04-10 17:09:50,946] [DEBUG] Loading corpus from path: valid.spacy
[2022-04-10 17:09:50,947] [DEBUG] Loading corpus from path: train.spacy
[2022-04-10 17:09:50,949] [DEBUG] Removed existing output directory: output_updated\model-best
[2022-04-10 17:09:50,950] [DEBUG] Removed existing output directory: output_updated\model-last


In [37]:
convert_dict = {0:'negatief', 1:'neutraal', 2:'positief'}

In [38]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp("goed gedaan makker")
print(convert_dict[np.argmax(list(demo.cats.values()))])

positief


In [39]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp("dat heb je bijzonder slecht gedaan, idioot")
print(convert_dict[np.argmax(list(demo.cats.values()))])

negatief
